In [1]:
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [3]:
grids = np.load('/fastpool/data/grid_post_fft.npy')

In [4]:
from matplotlib.colors import LogNorm

In [5]:
psf1 = grids[0].real / grids[0].real.max()

In [7]:
psf1.min()

-0.0004883886

In [39]:
%matplotlib widget
plt.figure(figsize=(5, 5))
plt.imshow(np.abs(psf1), norm=LogNorm(vmin=1e-6, vmax=1e-3))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
np.unravel_index(psf1.argmax(), shape=psf1.shape)

(4000, 4000)

Question for Bram
- How to divide out the FT of the kernel; other than...FFT out of the main code
- Which spheroidal to use?
- W term how?
- weighting how?
- which API to use?
- Normalization

In [9]:
grids.shape

(4, 8000, 8000)

Err what's the Fourier transform of a spheroidal?

In [10]:
from idg import util

Error importing OpenCL: ('/opt/lib/libidg-opencl.so: cannot open shared object file: No such file or directory',)


In [11]:
subgrid_size     = 32
grid_size        = 8000
spheroidal = util.get_example_spheroidal(subgrid_size)

In [12]:
%matplotlib widget
plt.imshow(spheroidal)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
%matplotlib widget
plt.plot(spheroidal[15,:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
from numpy import fft

In [15]:
s = fft.fft2(spheroidal)
s = fft.fftshift(s)
s1 = np.zeros((grid_size, grid_size), dtype = np.complex64)
support_size1 = (grid_size - subgrid_size) // 2
support_size2 = (grid_size + subgrid_size) // 2
s1[support_size1:support_size2, support_size1:support_size2] = s
s1 = fft.ifftshift(s1)

In [16]:
s1

array([[ 1.5199802e+02+0.j, -9.8025330e+01+0.j,  2.2604620e+01+0.j, ...,
        -6.1333245e-01+0.j,  2.2604620e+01+0.j, -9.8025330e+01+0.j],
       [-9.8025330e+01+0.j,  6.3217709e+01+0.j, -1.4577990e+01+0.j, ...,
         3.9554533e-01+0.j, -1.4577990e+01+0.j,  6.3217709e+01+0.j],
       [ 2.2604620e+01+0.j, -1.4577990e+01+0.j,  3.3616810e+00+0.j, ...,
        -9.1212697e-02+0.j,  3.3616810e+00+0.j, -1.4577990e+01+0.j],
       ...,
       [-6.1333245e-01+0.j,  3.9554533e-01+0.j, -9.1212697e-02+0.j, ...,
         2.4752594e-03+0.j, -9.1212697e-02+0.j,  3.9554533e-01+0.j],
       [ 2.2604620e+01+0.j, -1.4577990e+01+0.j,  3.3616810e+00+0.j, ...,
        -9.1212697e-02+0.j,  3.3616810e+00+0.j, -1.4577990e+01+0.j],
       [-9.8025330e+01+0.j,  6.3217709e+01+0.j, -1.4577990e+01+0.j, ...,
         3.9554533e-01+0.j, -1.4577990e+01+0.j,  6.3217709e+01+0.j]],
      dtype=complex64)

In [17]:
%matplotlib widget
plt.imshow(fft.fftshift(s1).real)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
s1_ft = fft.fft2(s1)

In [19]:
s1_ft.max()

(1023.9991360018892-4.437335915374874e-14j)

In [20]:
grids[0].max()

(38.931152+0j)

In [21]:
s1_ift = fft.ifft2(s1)

In [22]:
%matplotlib widget
plt.imshow(s1_ift.real)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
s1_ift.real.max()

1.5999986500029516e-05

In [24]:
grids[0].real.max() / s1_ift.real.max()

2433199.074491605

In [45]:
psf2 = (grids[0] / s1_ift.real).real
psf2 /= psf2[4000, 4000]

In [26]:
psf2

array([[ 1.75651725e-07,  6.13050864e-08, -8.54702166e-08, ...,
        -1.47756969e-07, -1.13950957e-07,  6.87928244e-08],
       [ 2.02770437e-07,  1.26084411e-07, -9.76356146e-09, ...,
        -1.93638119e-07, -1.05078605e-07,  9.30180846e-08],
       [ 2.09149141e-07,  1.61437545e-07,  3.87930409e-08, ...,
        -1.13617636e-07,  9.60449491e-10,  1.24093181e-07],
       ...,
       [ 1.16744508e-07,  1.01363915e-07,  1.82688917e-08, ...,
        -5.07421009e-08, -3.50565666e-08, -7.36415404e-08],
       [ 2.15119531e-07,  1.44115567e-07, -2.84153487e-09, ...,
        -5.76011678e-09,  1.83342711e-08,  5.57856078e-08],
       [ 2.09704866e-07,  8.86948805e-08, -7.84405461e-08, ...,
        -4.50606254e-08, -2.45972475e-08,  9.21577146e-08]])

In [48]:
%matplotlib widget
plt.figure(figsize=(5, 5))
plt.imshow(np.abs(psf2), norm=LogNorm(vmin=1e-6, vmax=1e-3))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [43]:
np.unravel_index(psf2.argmax(), shape=psf2.shape)

(0, 5228)